In [ ]:
import numpy as np
import os
import scipy.io

import holoviews as hv; hv.extension('bokeh',logo=False)

hv.opts.defaults(hv.opts.Raster(cmap="gray", xaxis=None, yaxis=None, frame_width=200))

In [ ]:
mat    = scipy.io.loadmat(os.path.join('..','DATA','allFaces.mat'))
faces  = mat['faces']
nfaces = mat['nfaces'].reshape(-1)

In [ ]:
## Function Definitions

def shrink(X,tau):
    Y = np.abs(X)-tau
    return np.sign(X) * np.maximum(Y,np.zeros_like(Y))
def SVT(X,tau):
    U,S,VT = np.linalg.svd(X,full_matrices=0)
    out = U @ np.diag(shrink(S,tau)) @ VT
    return out
def RPCA(X):
    n1,n2 = X.shape
    mu = n1*n2/(4*np.sum(np.abs(X.reshape(-1))))
    lambd = 1/np.sqrt(np.maximum(n1,n2))
    thresh = 10**(-7) * np.linalg.norm(X)
    
    S = np.zeros_like(X)
    Y = np.zeros_like(X)
    L = np.zeros_like(X)
    count = 0
    while (np.linalg.norm(X-L-S) > thresh) and (count < 1000):
        L = SVT(X-S+(1/mu)*Y,1/mu)
        S = shrink(X-L+(1/mu)*Y,lambd/mu)
        Y = Y + mu*(X-L-S)
        count += 1
    return L,S

In [ ]:
X = faces[:,:nfaces[0]]
L,S = RPCA(X)

In [ ]:
imgs = {}
for k in  (3,4,14,15,17,18,19,20,21,32,43):
    imgs[k] = (hv.Raster(np.reshape(X[:,k-1],(168,192)).T).opts(title='X')+\
               hv.Raster(np.reshape(L[:,k-1],(168,192)).T).opts(title='L')+\
               hv.Raster(np.reshape(S[:,k-1],(168,192)).T).opts(title='S')
              ).opts(shared_axes=False)
hv.HoloMap(imgs, kdims="case").redim.values( case= imgs.keys() ).opts(framewise=True, axiswise=True ).collate()